In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')
# Path to your subfolder containing the data
folder_path = '/content/drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


In [2]:
dataset_path1 = folder_path + "cleaned_data.csv"
dataset_path2 = folder_path + "one_hot_encoding.csv"

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Memuat data
df = pd.read_csv(dataset_path1)
one_hot = pd.read_csv(dataset_path2)

# Persiapan label untuk pelatihan
rating = [0, 4, 6, 8, 10]
new_label = ['bad', 'average', 'good', 'favourite']
new_label = {i: label for i, label in zip(rating, new_label)}
df["rating_label"] = df['vote_average'].map(new_label)

# Menghapus entri tanpa label
labeled = df.dropna(subset=['rating_label'])
merged = pd.merge(labeled, one_hot, on='title')

# Mengubah label ke dalam format numerik
mapping_label = {'bad': 0, 'average': 1, 'good': 2, 'favourite': 3}
merged['rating_label'] = merged['rating_label'].replace(mapping_label)

merged.to_csv('merged.csv')

# Tokenisasi dan padding
overview_data = merged['overview'].values
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(overview_data)
overview_sequences = tokenizer.texts_to_sequences(overview_data)
max_len = 100
overview_sequences = pad_sequences(overview_sequences, maxlen=max_len)

# Split data menjadi training dan testing
X = overview_sequences
y = merged['rating_label']
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, random_state=42)

# Definisikan model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_len,)),
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# Latih model
model.fit(train_X, train_y, epochs=10, batch_size=32, validation_data=(test_X, test_y))

# Simpan model
model.save('model_overview.h5')

# Konversi ke TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TensorFlow Lite
with open('model_overview.tflite', 'wb') as f:
    f.write(tflite_model)


Epoch 1/10
16566/16566 [==============================] - 243s 15ms/step - loss: 0.4377 - acc: 0.8867 - val_loss: 0.4011 - val_acc: 0.8982
Epoch 2/10
16566/16566 [==============================] - 180s 11ms/step - loss: 0.3629 - acc: 0.9063 - val_loss: 0.3697 - val_acc: 0.9055
Epoch 3/10
16566/16566 [==============================] - 172s 10ms/step - loss: 0.3322 - acc: 0.9135 - val_loss: 0.3653 - val_acc: 0.9074
Epoch 4/10
16566/16566 [==============================] - 173s 10ms/step - loss: 0.3056 - acc: 0.9205 - val_loss: 0.3510 - val_acc: 0.9138
Epoch 5/10
16566/16566 [==============================] - 172s 10ms/step - loss: 0.2829 - acc: 0.9265 - val_loss: 0.3414 - val_acc: 0.9167
Epoch 6/10
16566/16566 [==============================] - 172s 10ms/step - loss: 0.2636 - acc: 0.9308 - val_loss: 0.3416 - val_acc: 0.9190
Epoch 7/10
16566/16566 [==============================] - 172s 10ms/step - loss: 0.2453 - acc: 0.9349 - val_loss: 0.3400 - val_acc: 0.9212
Epoch 8/10
16566/16566 [===

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
